In [1]:
import numpy as np
import pandas as pd

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.2.2 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from nltk import pos_tag

In [5]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [6]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [7]:
import sklearn
from sklearn import model_selection

In [8]:
import re

In [9]:
import time

In [10]:
#Import Data
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')
#attributes=pd.read_csv('attributes.csv')
description=pd.read_csv('product_descriptions.csv')

In [11]:
train_data=train_data.drop('id',axis=1)

In [12]:
def clean_text(sentence):
    sentence = unicode(sentence,'utf-8', errors='replace')
    cleaned=' '.join([stemmer.stem(w) for w in sentence.split()])
    cleaned=' '.join([w for w in cleaned.split() if w not in stop])
    cleaned=re.sub('\W',' ',cleaned)
    cleaned=' '.join([w for w in cleaned.split() if w != 'x' and w!= 'in'])
    return cleaned

In [14]:
train_data.search_term=train_data.search_term.apply(clean_text)

In [15]:
description.product_description=description.product_description.apply(clean_text)


In [16]:
train_data.product_title=train_data.product_title.apply(clean_text)

In [17]:
train_data.head()

,product_uid,product_title,search_term,relevance
0,100001,simpson strong ti 12 gaug angl,angl bracket,3.00
1,100001,simpson strong ti 12 gaug angl,l bracket,2.50
2,100002,behr premium textur deckov 1 gal sc 141 tugboa...,deck,3.00
3,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33
4,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67


In [18]:
train_vec=pd.merge(train_data,description,left_on='product_uid',right_on='product_uid')

train_vec.head()

,product_uid,product_title,search_term,relevance,product_description
0,100001,simpson strong ti 12 gaug angl,angl bracket,3.00,onli angl make joint stronger also provid cons...
1,100001,simpson strong ti 12 gaug angl,l bracket,2.50,onli angl make joint stronger also provid cons...
2,100002,behr premium textur deckov 1 gal sc 141 tugboa...,deck,3.00,behr premium textur deckov innov solid color c...
3,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat bathroom delta vero single handl shower ...
4,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat bathroom delta vero single handl shower ...


In [19]:
#spell correcting

from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter('\n'.join(s for s in train_vec.product_description.as_matrix().flatten()).split())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [20]:
train_vec.search_term=train_vec.search_term.apply(lambda x: ' '.join([correction(w) for w in x.split()]))

In [21]:
train_vec.to_csv('myplayingwithdataset.csv')

In [22]:
train_vec.shape

(74067, 5)

In [23]:
to_train=pd.DataFrame(np.zeros([len(train_vec),5]),columns=['Title_1','Desc_1','Title_2','Desc_2','Len'])

In [24]:
def shrink(k):
    return 1-1/np.exp(k)

In [25]:
def my_docu_in_search(sentence, docu):
    wf=[]
    for word in sentence.split():
        wf.append((np.array(docu.split())==[word]).sum()/1.0/len(docu.split()))
    return np.array(wf).sum()

In [26]:
def my_search_in_docu(sentence, docu):
    wf=[]
    for word in sentence.split():
        wf.append((np.array(docu.split())==[word]).sum())
    return shrink(np.array(wf)).mean()

In [312]:
'''
for i in range(len(train_vec)):
    to_train['Title'][i]=my_tf_df(train_vec['search_term'][i],train_vec['product_title'][i])
    to_train['Desc'][i]=my_tf_df(train_vec['search_term'][i],train_vec['product_description'][i])
    to_train['Len'][i]=len(train_vec['search_term'][i].split())
'''

In [27]:
for i in range(len(train_vec)):
    to_train['Title_1'][i]=my_docu_in_search(train_vec['search_term'][i],train_vec['product_title'][i])
    to_train['Desc_1'][i]=my_docu_in_search(train_vec['search_term'][i],train_vec['product_description'][i])
    to_train['Title_2'][i]=my_search_in_docu(train_vec['search_term'][i],train_vec['product_title'][i])
    to_train['Desc_2'][i]=my_search_in_docu(train_vec['search_term'][i],train_vec['product_description'][i])
    to_train['Len'][i]=len(train_vec['search_term'][i].split())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice.
  """
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [28]:
to_train['uid']=train_vec['product_uid']

In [29]:
import xgboost as xgb

In [30]:
param={}
param['eta']=0.01
param['max_depth']=6
param['silent']=1
param['eval_metric']='rmse'
param['min_child_weight']=3
param['subsample']=0.7
param['colsample_bytree']=0.7
num_rounds=50000

In [31]:
start_ = time.time()

#x_train = np.array(train_vec.iloc[0:50000])
#y_train = np.array(Relevance.iloc[0:50000])

#x_validation = np.array(train_vec.iloc[50000:])
#y_validation = np.array(Relevance.iloc[50000:])
x_train, x_validation, y_train, y_validation=model_selection.train_test_split(to_train,train_vec['relevance'],test_size=0.3)

xgtrain = xgb.DMatrix(x_train, label= y_train)
xgvalidation=xgb.DMatrix(x_validation,label=y_validation)

In [32]:
clf = xgb.train(param, xgtrain, num_rounds,evals=[ (xgtrain,'train'),(xgvalidation,'eval')],
                early_stopping_rounds=100, verbose_eval =100)

[0]	train-rmse:1.93746	eval-rmse:1.93792
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 100 rounds.
[100]	train-rmse:0.839695	eval-rmse:0.840028
[200]	train-rmse:0.540075	eval-rmse:0.542403
[300]	train-rmse:0.483827	eval-rmse:0.488405
[400]	train-rmse:0.474139	eval-rmse:0.480557
[500]	train-rmse:0.471425	eval-rmse:0.479447
[600]	train-rmse:0.469888	eval-rmse:0.479246
[700]	train-rmse:0.468619	eval-rmse:0.479234
Stopping. Best iteration:
[633]	train-rmse:0.469459	eval-rmse:0.479226

